In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import time
import os
import pandas
import requests
import boto3
import numpy as np
import xarray as xr
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
import rioxarray

In [ ]:
# get credentials
s3_cred_endpoint = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'
def get_temp_creds():
    temp_creds_url = s3_cred_endpoint
    return requests.get(temp_creds_url).json()

temp_creds_req = get_temp_creds()

session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')

In [ ]:
session

In [ ]:
rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

In [ ]:
# read the .csv file with S3 links
stack_df = pandas.read_csv('/home/jovyan/nch21_hls_timeseries/HLS_data/T13TDE/stack.csv')
stack_df = stack_df.loc[~stack_df['date'].isna(), :]
stack_df.reset_index(inplace=True)
stack_df

In [ ]:
# subset the s3 links by band
header = ['label', 'L30_band', 'S30_band', 'read']
data = [
    ['coastal_aerosol', 'B01', 'B01', False],
    ['blue', 'B02', 'B02', True],
    ['green', 'B03', 'B03', True],
    ['red', 'B04', 'B04', True],
    ['red-edge_1', None, 'B05', False],
    ['red-edge_2', None, 'B06', False],
    ['red-edge_3', None, 'B07', False],
    ['nir_broad', None, 'B08', False],
    ['nir', 'B05', 'B8A', True],
    ['swir_1', 'B06', 'B11', True],
    ['swir_2', 'B07', 'B12', True],
    ['water_vapor', None, 'B09', False],
    ['cirrus', 'B09', 'B10', False],
    ['thermal_infrared_1', 'B10', None, False],
    ['thermal_infrared_2', 'B11', None, False],
    ['fmask', 'Fmask', 'Fmask', True]
]

band_df = pandas.DataFrame(data, columns=header)
band_df

In [ ]:
%%time

chunks=dict(band=1, x=256, y=256)

hls_ds = None

for i in range(0, band_df.shape[0]):
    if band_df.loc[i, 'read'] == True:
        # subset stack for links for each band
        band_stack = stack_df.loc[
        ((stack_df['band'] == band_df.loc[i,'L30_band']) & (stack_df['sensor'] == 'L30')) |
        ((stack_df['band'] == band_df.loc[i,'S30_band']) & (stack_df['sensor'] == 'S30')), :]
        
        # create the time index
        band_time = [datetime.strptime(str(t), '%Y%jT%H%M%S') for t in band_stack['date']]
        xr.Variable('time', band_time)

        #s3_links = band_stack['S3_links']
        s3_links = band_stack['local_links']
        
        # get the band label
        band_label = band_df.loc[i, 'label']
        
        # open the links
        hls_ts_da = xr.concat([rioxarray.open_rasterio(f, chunks=chunks).squeeze('band', drop=True) for f in s3_links], dim=band_time)
        hls_ts_da = hls_ts_da.rename({'concat_dim':'time'})
        
        if hls_ds is None:
            hls_ds = xr.Dataset({band_label: hls_ts_da})
        else:
            hls_ds[band_label] = hls_ts_da

#hls_ds.rename({'concat_dim':'time'})
hls_ds

In [ ]:
%%time

def SI(b1, b2):
    si = (b1 - b2) / (b1 + b2)
    si = xr.where(si < -1.0, -1.0, si)
    si = xr.where(si > 1.0, 1.0, si)
    si = xr.where(np.isfinite(si), si, np.nan)
    return(si)

# calculate NDVI (normalized difference vegetation index)
hls_ds['ndvi'] = SI(hls_ds['nir'], hls_ds['red'])

# calculate NBR (normalized difference burn ratio)
hls_ds['nbr'] = SI(hls_ds['nir'], hls_ds['swir_2'])

# calculate NDWI (normalized difference water index)
hls_ds['ndwi_gao'] = SI(hls_ds['nir'], hls_ds['swir_1'])
hls_ds['ndwi_mcfeeters'] = SI(hls_ds['green'], hls_ds['nir'])

# calculate NDSI (normalized difference snow index)
hls_ds['ndsi'] = SI(hls_ds['green'], hls_ds['swir_1'])

hls_ds

In [ ]:
da = hls_ds['nbr']

roll = da.rolling(time = 5, min_periods=5, center = True)
roll



In [ ]:
test = roll.mean(keep_attrs=False)
test

In [ ]:
import geoviews as gv
import hvplot.xarray

# nbr_i.hvplot(x='x', y='y', cmap='viridis', title='NBR') + \
# roll_nbr_i.hvplot(x='x', y='y', cmap='viridis', title='roll NBR')



In [ ]:
i=20
nbr_i = da[i, :, :]
nbr_i

In [ ]:
roll_nbr_i = test[i, :, :]
roll_nbr_i

In [ ]:
#roll_nbr_i.hvplot(x='x', y='y', cmap='viridis', title='roll NBR')


In [ ]:
#nbr_i.hvplot(x='x', y='y', cmap='viridis', title='NBR')